## Variant Calling

#### `Author: Simon Hackl`
#### `Project: The OMPeome of Treponema pallidum`
#### `Contact: simon.hackl@uni-tuebingen.de`
#### `Date: 15.02.2022`

This _Python_ Notebook guides through and documents the steps of variant calling.

### 1. Acquisition of Reference Genome and Sample Data
The chosen reference genome sequence and annotation were downloaded from https://www.ncbi.nlm.nih.gov/nuccore/NC_021490.2 and stored at `./R4_TPOMPeome_Hackl2022_VariantCalling/ReferenceGenome/NC_021490.2.fasta` and `./R4_TPOMPeome_Hackl2022_VariantCalling/ReferenceGenome/NC_021490.2.gff3`.

A set of $74$ DNA sequencing read datasets already processed with `Clip&Merge` were collected from the _romanov_ server of the _Integrative Transcriptomics_ group and stored in the `./R4_TPOMPeome_Hackl2022_VariantCalling/SampleReads/` directory.

### 2. Variant Calling

Genotyping was conducted in three modes for each sample by invoking the bash script below in the directory `./R4_TPOMPeome_Hackl2022_VariantCalling/`:
    
    1. Genotyping with `GATK UnifiedGenotyper` without RecoverMI applied ().
       Filename: <Samplename>-NC_021490.2-UG-variants.vcf
       
    2. Genotyping with `GATK UnifiedGenotyper` with RecoverMI applied.
       Filename: <Samplename>-NC_021490.2-UG-variants.fxd.vcf
       
    3. Genotyping with `GATK HaplotypeCaller` with RecoverMI applied.
       Filename: <Samplename>-NC_021490.2-HC-variants.fxd.vcf
    
In advance, `DeDup` was applied to true sample reads, i.e. those reads that were derived from genomes were not treated. The resulting vcf files were stored at `./R4_TPOMPeome_Hackl2022_VariantCalling/VariantCalling` with the file names as explained above.

**NOTE:** In order to run the script, `bwa`, `samtools`, `GATK` and `DeDup` have to be installed and accessible via the path variables. `RecoverMI.py` has to be stored in the `./R4_TPOMPeome_Hackl2022_VariantCalling/VariantCalling` directory.

```
#!/bin/bash

# Create output directory.
mkdir ./VariantCalling/

# Generate reference genome indices.
bwa index ./ReferenceGenome/NC_021490.2.fasta
samtools faidx ./ReferenceGenome/NC_021490.2.fasta
samtools dict ./ReferenceGenome/NC_021490.2.fasta > ./ReferenceGenome/NC_021490.2.dict

# Run deduplication, multi-map preservation and variant calling per sample.
for sampleDir in ./SampleReads/*
do
	sampleName=$(basename "${sampleDir}")
	if [ ! -d "./VariantCalling/${sampleName}/" ]; then
		mkdir ./VariantCalling/${sampleName}
		echo ${sampleName}
		for sampleFASTQFile in ./SampleReads/$sampleName/*.fq.gz
		do
			bwa mem ./ReferenceGenome/NC_021490.2.fasta ${sampleFASTQFile} -t 8 -a -R '@RG\tID:1\tPL:ILLUMINA\tLB:lib1\tSM:A2_1' | samtools sort -@ 8 -o ./VariantCalling/${sampleName}/mapped.bam
			# If samples are reads derived from genomes do not apply DeDup.
			if [[ ${sampleName} == "BosniaA" || ${sampleName} == "CDC2" || ${sampleName} == "Chicago" || ${sampleName} == "Dallas" || ${sampleName} == "Fribourg" || ${sampleName} == "Gauthier" || ${sampleName} == "MexicoA" || ${sampleName} == "Nichols" || ${sampleName} == "SamoaD" || ${sampleName} == "Seattle81" || ${sampleName} == "SS14" || ${sampleName} == "Cuniculi" ]]; then
				samtools index -@ 8 ./VariantCalling/${sampleName}/mapped.bam
				# Run variant valling with UnifiedGenotyper on mapping without preserved multi-mappings.
				gatk -T UnifiedGenotyper -R ./ReferenceGenome/NC_021490.2.fasta -I ./VariantCalling/${sampleName}/mapped.bam -nt 4 -out_mode EMIT_ALL_SITES -o ./VariantCalling/${sampleName}/${sampleName}-NC_021490.2-UG-variants.vcf

				# Sort by read name and apply RecoverMI.
				samtools sort -n -@ 8 -o ./VariantCalling/${sampleName}/mapped.sorted.sam ./VariantCalling/${sampleName}/mapped.bam
				rm ./VariantCalling/${sampleName}/mapped.bam
				rm ./VariantCalling/${sampleName}/mapped.bam.*
				./RecoverMI.py ./VariantCalling/${sampleName}/mapped.sorted.sam ./VariantCalling/${sampleName}/
				rm ./VariantCalling/${sampleName}/mapped.sorted.sam
				# Convert sam to bam, re-sort by position and index bam file.
				samtools sort -o ./VariantCalling/${sampleName}/mapped.sorted.fxd.bam -@ 8 ./VariantCalling/${sampleName}/mapped.sorted.fxd.sam
				samtools index -@ 8 ./VariantCalling/${sampleName}/mapped.sorted.fxd.bam

				# Run variant calling with HaplotypeCaller on mapping with preserved multi-mappings.
				gatk -T HaplotypeCaller -R ./ReferenceGenome/NC_021490.2.fasta -I ./VariantCalling/${sampleName}/mapped.sorted.fxd.bam -o ./VariantCalling/${sampleName}/${sampleName}-NC_021490.2-HC-variants.fxd.vcf
				# Run variant valling with UnifiedGenotyper on mapping with preserved multi-mappings.
				gatk -T UnifiedGenotyper -R ./ReferenceGenome/NC_021490.2.fasta -I ./VariantCalling/${sampleName}/mapped.sorted.fxd.bam -nt 4 -out_mode EMIT_ALL_SITES -o ./VariantCalling/${sampleName}/${sampleName}-NC_021490.2-UG-variants.fxd.vcf 
			else
				# Run DeDup to remove PCR duplicates.
				dedup -i ./VariantCalling/${sampleName}/mapped.bam -o ./VariantCalling/${sampleName}/
				mv ./VariantCalling/${sampleName}/mapped_rmdup.bam ./VariantCalling/${sampleName}/mapped.rmdup.bam
				rm ./VariantCalling/${sampleName}/mapped.bam
		
				samtools sort -@ 4 -o ./VariantCalling/${sampleName}/mapped.rmdup.sorted.bam ./VariantCalling/${sampleName}/mapped.rmdup.bam
				samtools index -@ 4 ./VariantCalling/${sampleName}/mapped.rmdup.sorted.bam
				# Run variant valling with UnifiedGenotyper on mapping without preserved multi-mappings.
				gatk -T UnifiedGenotyper -R ./ReferenceGenome/NC_021490.2.fasta -I ./VariantCalling/${sampleName}/mapped.rmdup.sorted.bam -nt 4 -out_mode EMIT_ALL_SITES -o ./VariantCalling/${sampleName}/${sampleName}-NC_021490.2-UG-variants.vcf

				# Sort by read name and apply RecoverMI.
				samtools sort -@ 4 -n -o ./VariantCalling/${sampleName}/mapped.rmdup.sortedNames.sam ./VariantCalling/${sampleName}/mapped.rmdup.sorted.bam
				rm ./VariantCalling/${sampleName}/mapped.rmdup.sorted.bam
				rm ./VariantCalling/${sampleName}/mapped.rmdup.sorted.bam.*
				./Scripts/RecoverMI.py ./VariantCalling/${sampleName}/mapped.rmdup.sortedNames.sam ./VariantCalling/${sampleName}/
				rm ./VariantCalling/${sampleName}/mapped.rmdup.sortedNames.sam
				## Convert sam to bam, re-sort by position and index bam file.
				samtools sort -o ./VariantCalling/${sampleName}/mapped.rmdup.sorted.fxd.bam -@ 4 ./VariantCalling/${sampleName}/mapped.rmdup.sortedNames.fxd.sam
				samtools index -@ 4 ./VariantCalling/${sampleName}/mapped.rmdup.sorted.fxd.bam

				# Run variant calling with HaplotypeCaller on mapping with preserved multi-mappings.
				gatk -T HaplotypeCaller -R ./ReferenceGenome/NC_021490.2.fasta -I ./VariantCalling/${sampleName}/mapped.rmdup.sorted.fxd.bam -o ./VariantCalling/${sampleName}/${sampleName}-NC_021490.2-HC-variants.fxd.vcf
				# Run variant valling with UnifiedGenotyper on mapping with preserved multi-mappings.
				gatk -T UnifiedGenotyper -R ./ReferenceGenome/NC_021490.2.fasta -I ./VariantCalling/${sampleName}/mapped.rmdup.sorted.fxd.bam -nt 4 -out_mode EMIT_ALL_SITES -o ./VariantCalling/${sampleName}/${sampleName}-NC_021490.2-UG-variants.fxd.vcf 
			fi
		break
		done
	fi
done
```

### 3. Assessing Repeat Resolution

In order to assess if `RecoverMI` worked correctly, the respective vcf files generated by `GATK UnifiedGenotyper` with and without prior application of `RecoverMI` were compared by calculating (wrt. gene length) the percentage of positions that were disregarded and actually located in a repeat.

For this, repeats were first identified using `DACCOR`.

### 3.1 Identification of Min. Read Length
In order to choose appropriate parameters for `DACCOR` the minimum read length of the dataset was extracted from the respective FASTQ files by using the bash script below (in the `./R4_TPOMPeome_Hackl2022_VariantCalling/VariantCalling` directory).

```
#!/bin/sh
for readFile in ./SampleReads/*/*.fq.gz
do
	echo "Processing: ${readFile}"
	zcat ${readFile} | awk '{if(NR%4==2) print length($1)}' | sort -n | uniq -c >> ./rl.txt
done
sort -n ./rl.txt | uniq -c >> ./UniqueReadLengths.txt
rm ./rl.txt
```

From the `UniqueReadLengths.txt` file the shortest read length was determined to be $30$.

### 3.2 Extraction of Gene Sequences

 A list with the names of the OMP coding genes of interest was created at `./R4_TPOMPeome_Hackl2022_VariantCalling/TPOMPGeneNames.txt`.

The respective locus and strand orientation of the genes in the chosen reference was detected by matching a TPXXXX gene name with the old_locus_tag attributes value (format TPANIC\_XXXX) from the reference genome annotation.

Next, the genes nucleotide sequences were extracted from the reference genome acoording to this information. The obtained sequences were stored at `./R4_TPOMPeome_Hackl2022_VariantCalling/TPOMPGenesSequences/`.

In [ ]:
import os
import shutil
import json

from subprocess import Popen, PIPE

import numpy as np
import seaborn as sns
import matplotlib.pylab as plt

plt.rcParams.update( { "text.usetex": False, "font.family": "serif" } )

In [ ]:
# Parse information from the genome annotation and store it in a dictionary accessible by the old_locus_tag attribute.
annotatedGenes = { }
with open( "./R4_TPOMPeome_Hackl2022_VariantCalling/ReferenceGenome/NC_021490.2.gff3", "r" ) as annotatedGenesFile :
    line = annotatedGenesFile.readline( )
    while line :
        if line.startswith( "##" ) : # Skip comment lines
            line = annotatedGenesFile.readline( )
            continue
        else :
            splitLine = line.split( "\t" )
            if splitLine[ 0 ] == "NC_021490.2" and splitLine[ 1 ] == "RefSeq" and ( splitLine[ 2 ] == "gene" or splitLine[ 2 ] == "pseudogene" )  :
                attributes = splitLine[ 8 ].strip( ).split( ";" )
                for attribute in attributes :
                    key, value = attribute.split( "=" )
                    if key == "old_locus_tag" :
                        annotatedGenes[ value ] = {
                            "locusStart": int( splitLine[ 3 ] ),
                            "locusEnd": int( splitLine[ 4 ] ),
                            "strandOrientation": splitLine[ 6 ]
                        }
        line = annotatedGenesFile.readline( )
        
# Parse the reference genome sequence.
referenceGenome = ""
with open( "./R4_TPOMPeome_Hackl2022_VariantCalling/ReferenceGenome/NC_021490.2.fasta", "r" ) as referenceGenomeFile :
    line = referenceGenomeFile.readline( ) # Skip fasta header
    line = referenceGenomeFile.readline( )
    while line :
        referenceGenome += line.strip( )
        line = referenceGenomeFile.readline( )
        
# For each of the OMP coding genes of interest the respective gene sequence is extracted.
if not os.path.isdir( "./R4_TPOMPeome_Hackl2022_VariantCalling/TPOMPGenesSequences/" ) :
    os.mkdir( "./R4_TPOMPeome_Hackl2022_VariantCalling/TPOMPGenesSequences/" )
with open( "./R4_TPOMPeome_Hackl2022_VariantCalling/TPOMPGeneNames.txt", "r" ) as OMPGenesFile :
    line = OMPGenesFile.readline( )
    while line :
        geneName = line.strip( )
        print( "Processing Gene: " + geneName )
        geneOldLocusTag = "TPANIC_" + geneName[ 2: ]
        geneStart = annotatedGenes[ geneOldLocusTag ][ "locusStart" ]
        geneEnd = annotatedGenes[ geneOldLocusTag ][ "locusEnd" ]
        geneOrientation = annotatedGenes[ geneOldLocusTag ][ "strandOrientation" ]
        geneSequence = referenceGenome[ geneStart - 1 : geneEnd ]
        with open( "./R4_TPOMPeome_Hackl2022_VariantCalling/TPOMPGenesSequences/" + geneName + ".fasta", "w+" ) as OMPGeneSequenceFile :
            OMPGeneSequenceFile.write( ">" + geneName + "\n" )
            sequenceChunks = [ geneSequence[ i : i + 80 ] for i in range( 0, len( geneSequence ), 80 ) ]
            for sequenceChunk in sequenceChunks :
                OMPGeneSequenceFile.write( sequenceChunk + "\n" )
        line = OMPGenesFile.readline( )

### 3.3 Identify Repeats with DACCOR identify

To identify repeats with `DACCOR` on gene and genome level the following bash script was run in `./R4_TPOMPeome_Hackl2022_VariantCalling/`.

**NOTE:** In order to run the script, `DACCOR` has to be installed and accessible via the path variables.

```
#!/bin/sh
mkdir ./DACCORIdentifyResults/
echo '-- Run DACCOR identify on single OMP genes'
FILES="./TPOMPGenesSequencess/TP*.fasta"
for file in $FILES
do
	echo "- Processing file: $file"
	fileBasename=`basename ${file%.fasta}`
	mkdir ./DACCORIdentifyResults/${fileBasename}
	outDir=./DACCORIdentifyResults/${fileBasename}
	daccor identify -i ${file} -o ${outDir} -m 0 -M 0 -rl 30 -k 15 -ws
done

echo '-- Run identify subprogram on full genome NC_021490.2'
file="./ReferenceGenome/NC_021490.2.fasta"
outDir=./DACCORIdentifyResults/NC_021490.2
daccor identify -i ${file} -o ${outDir} -m 0 -M 0 -rl 30 -k 15 -ws'
```

### 3.4 Identify Repeats Interfering with Gene Loci
Next, in order to treat only repeats that truely interfer with gene positions the OMP genes positions are compared with the repeat positions and, for each gene, the percentage of gene positions that are actual repeat positions is computed.

In [ ]:
geneNames = list( )
with open( "./R4_TPOMPeome_Hackl2022_VariantCalling/TPOMPGeneNames.txt", "r" ) as OMPGenesFile :
    line = OMPGenesFile.readline( )
    while line :
        geneNames.append( line.strip( ) )
        line = OMPGenesFile.readline( )
        
# Parse identified repeats.
repeats = { }
for directory in os.listdir( "./R4_TPOMPeome_Hackl2022_VariantCalling/DACCORIdentifyResults/" ) :
    for file in os.listdir( "./R4_TPOMPeome_Hackl2022_VariantCalling/DACCORIdentifyResults/" + directory + "/" ) :
        if file.endswith( "_Summary.csv" ) :
            isRepeatInfo = False
            with open( "./R4_TPOMPeome_Hackl2022_VariantCalling/DACCORIdentifyResults/" + directory + "/" + file, "r" ) as repeatsSummary :
                line = True
                isRepeatInfo = False
                while line :
                    line = repeatsSummary.readline( )
                    if line.startswith( "fastaName\t" ) :
                        isRepeatInfo = True
                        continue
                    if isRepeatInfo and len( line.split( "\t") ) > 1 :
                        offset = 0
                        if directory.startswith( "TP" ) :
                            offset = annotatedGenes[ "TPANIC_" + directory[ 2: ] ][ 'locusStart' ] - 1
                        if line.startswith( "NC_" ) : # Treat _ in reference name with index shift.
                            repeats[ directory + "-" + line.split( "\t" )[ 1 ] ] = {
                                "length": int( line.split( "\t" )[ 0 ].split( "_" )[ 5 ] ),
                                "starts": [ int( s ) for s in line.split( "\t" )[ 0 ].split( "_" )[ 3 ].strip( "]" ).strip( "[" ).split( "," ) ]
                            }
                        else :
                            repeats[ directory + "-" + line.split( "\t" )[ 1 ] ] = {
                                "length": int( line.split( "\t" )[ 0 ].split( "_" )[4] ),
                                "starts": [ int( s ) + offset for s in line.split( "\t" )[ 0 ].split( "_" )[2].strip( "]" ).strip( "[" ).split( "," ) ]
                            }
                            
relevantRepeats = { }
affectedGenes = { }
for geneName in geneNames :
    geneInfo = annotatedGenes[ "TPANIC_" + geneName[ 2: ] ]
    genePositions = list( range( geneInfo[ 'locusStart' ], geneInfo[ 'locusEnd' ] + 1 ) )
    geneLength = geneInfo[ 'locusEnd' ] - geneInfo[ 'locusStart' ] + 1
    totalPositions = 0
    for repeatName, repeatInfo in repeats.items( ) :
        commonPositions = 0
        for repeatStart in repeatInfo[ 'starts' ] :
            repeatPositions = list( range( repeatStart, repeatStart + repeatInfo[ 'length' ] - 1 ) )
            for gp in genePositions :
                for rp in repeatPositions :
                    if gp == rp :
                        commonPositions += 1
                        relevantRepeats[ repeatName ] = repeats[ repeatName ]
        totalPositions += commonPositions
        if commonPositions > 0 :
            if not geneName in affectedGenes :
                affectedGenes[ geneName ] = { }
            if not repeatName in affectedGenes[ geneName ] :
                affectedGenes[ geneName ][ repeatName ] = [ ]
            affectedGenes[ geneName ][ repeatName ].append( round( ( commonPositions / geneLength ) * 100, 4 ) )
    if geneName in affectedGenes :
        if not "Total" in affectedGenes[ geneName ] :
            affectedGenes[ geneName ][ "Total" ] = [ ]
        affectedGenes[ geneName ][ "Total" ].append( min( 100.0, round( ( totalPositions / geneLength ) * 100, 4 ) ) )
        
affectedGeneNames = sorted( list( affectedGenes.keys( ) ) )
relevantRepeatNames = list( relevantRepeats.keys( ) )
relevantRepeatNames.sort( key = lambda rN : int( rN.split( "_" )[ 1 ] ) )
relevantRepeatNamesANSorted = [ ]
for rN in relevantRepeatNames :
    if rN.startswith( "NC" ) :
        relevantRepeatNamesANSorted.append( rN )
for rN in relevantRepeatNames :
    if rN.startswith( "TP" ) :
        relevantRepeatNamesANSorted.append( rN )
relevantRepeatNames = relevantRepeatNamesANSorted
data2D = [ ]
for relevantRepeatName in relevantRepeatNames :
    data1D = [ ]
    for affectedGeneName in affectedGeneNames :
        if relevantRepeatName in affectedGenes[ affectedGeneName ] :
            data1D.append( sum( affectedGenes[ affectedGeneName ][ relevantRepeatName ] ) )
        else :
            data1D.append( 0 )
    data2D.append( data1D )
total1D = [ ]
for affectedGeneName in affectedGeneNames :
    total1D.append( sum( affectedGenes[ affectedGeneName ][ "Total" ] ) )
data2D.append( total1D )

fig, ax = plt.subplots( figsize = ( 12, 10 ) )
res = sns.heatmap( data2D,
             cmap = "GnBu",
             cbar_kws = {
                 "aspect": 50,
                 "label": "Overlapping Positions (%)",
                 "orientation": "horizontal"
             },
             vmin = 0,
             vmax = 100,
             center = 50,
             linewidths = 1,
             annot = True,
             annot_kws = {
                 # 'rotation': 90,
                 'va': 'center',
                 'ha': 'center'
             }
           )
for t in res.texts:
    if t.get_text( ) == "0" :
        t.set_text( "" )
    elif t.get_text( ) == "1e+02" :
        t.set_text( "100 %" )
    else :
        t.set_text( t.get_text( ) + " %" )
ax.set_yticklabels( [ rN.split( "-" )[ 0 ].replace( "NC021490.2", "" ) + " " + rN.split( "-" )[ 1 ].replace( "_", " " ).replace( "r", "R" ) for rN in relevantRepeatNames ] + [ "Total" ], size = 12 )
ax.set_ylabel( "Repeats", size = 18 )
ax.set_xticklabels( affectedGeneNames, size = 14 )
ax.set_xlabel( "Genes", size = 18 )
plt.setp( ax.get_xticklabels( ), rotation = 55, ha = "right", rotation_mode = "anchor")
plt.setp( ax.get_yticklabels( ), rotation = 0, ha = "right", rotation_mode = "anchor")
ax.set_title( "Overlapping Gene and Repeat Positions", size = 20 )
ax.figure.axes[-1].xaxis.label.set_size( 18 )
ax.figure.axes[-1].tick_params( labelsize = 14 )
fig.tight_layout( )
plt.show( )

### 3.5 Assess Disregarded Repeat Positions per Gene per Sample (without RecoverMI)

In [ ]:
# First assess the overlapping positions per gene and per repeat.
perGeneRepeatPositions = { }
for geneName in geneNames :
    geneInfo = annotatedGenes[ "TPANIC_" + geneName[ 2: ] ]
    genePositions = list( range( geneInfo[ 'locusStart' ], geneInfo[ 'locusEnd' ] + 1 ) )
    geneLength = geneInfo[ 'locusEnd' ] - geneInfo[ 'locusStart' ] + 1
    for repeatName, repeatInfo in repeats.items( ) :
        commonPositions = set( )
        for repeatStart in repeatInfo[ 'starts' ] :
            repeatPositions = list( range( repeatStart, repeatStart + repeatInfo[ 'length' ] - 1 ) )
            for gp in genePositions :
                for rp in repeatPositions :
                    if gp == rp :
                        commonPositions.add( gp )
        if len( commonPositions ) != 0 :
            perGeneRepeatPositions[ geneName ] = commonPositions
            
# For each sample, assess the number of uncalled positions per gene and per repeat (for untreated .vcf).
perSamplePerGeneUncalledCountUntreated = { }
for sampleName in os.listdir( "./R4_TPOMPeome_Hackl2022_VariantCalling/VariantCalling/" ) :
    perSamplePerGeneUncalledCountUntreated[ sampleName ] = { }
    with open( "./R4_TPOMPeome_Hackl2022_VariantCalling/VariantCalling/" + sampleName + "/" + sampleName + "-NC_021490.2-UG-variants.vcf", "r" ) as untreatedCalls :
        line = untreatedCalls.readline( )
        while line :
            if not line.startswith( "#" ) :
                position = int( line.split( "\t" )[ 1 ] )
                for geneName in geneNames :
                    if geneName in perGeneRepeatPositions.keys( ) and not geneName in perSamplePerGeneUncalledCountUntreated[ sampleName ] :
                        perSamplePerGeneUncalledCountUntreated[ sampleName ][ geneName ] = 0
                    if geneName in perGeneRepeatPositions.keys( ) and position in perGeneRepeatPositions[ geneName ] and line.split( "\t" )[ 9 ].startswith( "./." ) :
                        perSamplePerGeneUncalledCountUntreated[ sampleName ][ geneName ] += 1
            line = untreatedCalls.readline( )
            
from matplotlib.ticker import MultipleLocator

In [ ]:
# Compute the percentage of uncalled positions per gene and per repeat and plot (for untreated .vcf).
fig, axs = plt.subplots( 4, 3, figsize = ( 11, 15 ), sharex = True, sharey = True )
axIdx0 = -1
axIdx1 = 0
for geneName in sorted( list( list( perSamplePerGeneUncalledCountUntreated.values( ) )[ 0 ].keys( ) ) ) :
    axIdx0 += 1
    if axIdx0 > 3 :
        axIdx0 = 0
        axIdx1 += 1
    percentages = [ ]
    ax = axs[ axIdx0 ][ axIdx1 ]
    for sampleName in sorted( list( perSamplePerGeneUncalledCountUntreated.keys( ) ) ) :
        geneInfo = annotatedGenes[ "TPANIC_" + geneName[ 2: ] ]
        genePositions = list( range( geneInfo[ 'locusStart' ], geneInfo[ 'locusEnd' ] + 1 ) )
        geneLength = geneInfo[ 'locusEnd' ] - geneInfo[ 'locusStart' ] + 1
        percentages.append(
            round( 100 * ( perSamplePerGeneUncalledCountUntreated[ sampleName ][ geneName ] / geneLength ), 4 )
        )
    ax.hist( percentages, bins = [ v + 1*10**-6 for v in np.arange( -5, 105, 5 ) ], color = "dimgray", rwidth = 0.9 )
    ax.grid( )
    ax.set_title( geneName, size = 18 )
    if axIdx0 == 3 :
        ax.set_xticks( np.arange( 0, 120, 20 ) )
        ax.xaxis.set_tick_params( labelsize = 14 )
        ax.set_xlabel( "Disregarded Positions [%]", size = 18 )
    ax.xaxis.set_minor_locator(MultipleLocator(5))
    if axIdx1 == 0 :
        #ax.set_yticks( np.arange( 0.5, 74.5 ) )
        ax.yaxis.set_tick_params( labelsize = 14 )
        ax.set_ylabel( "Count", size = 18 )
plt.suptitle( "Histograms of Disregarded Repeat Positions in 74 Samples per Gene\nRecoverMI not applied", y = 1.01, size = 20 )
plt.tight_layout( )
plt.show( )

In [ ]:
# Compute the percentage of uncalled positions per gene and per repeat and plot (for untreated .vcf).
data2D = [ ]
for sampleName in sorted( list( perSamplePerGeneUncalledCountUntreated.keys( ) ) ) :
    data1D = [ ]
    for geneName in sorted( list( list( perSamplePerGeneUncalledCountUntreated.values( ) )[ 0 ].keys( ) ) ) :
        geneInfo = annotatedGenes[ "TPANIC_" + geneName[ 2: ] ]
        genePositions = list( range( geneInfo[ 'locusStart' ], geneInfo[ 'locusEnd' ] + 1 ) )
        geneLength = geneInfo[ 'locusEnd' ] - geneInfo[ 'locusStart' ] + 1
        data1D.append(
            round( 100 * ( perSamplePerGeneUncalledCountUntreated[ sampleName ][ geneName ] / geneLength ), 4 )
        )
    data2D.append( data1D )

fig, ax = plt.subplots( figsize = ( 12, 18 ) )
res = sns.heatmap( data2D,
     cmap = "GnBu",
     cbar_kws = {
         "aspect": 50,
         "label": "Disregarded Positions (%)",
         "orientation": "horizontal",
         "pad": 0.08
     },
     vmin = 0,
     vmax = 100,
     center = 50,
     linewidths = 1,
     annot = True,
     annot_kws = {
         'va': 'center',
         'ha': 'center'
     }
)
for t in res.texts:
    if t.get_text( ) == "0" :
        t.set_text( "" )
    elif t.get_text( ) == "1e+02" :
        t.set_text( "100 %" )
    else :
        t.set_text( t.get_text( ) + " %" )
ax.set_xticklabels( sorted( list( list( perSamplePerGeneUncalledCountUntreated.values( ) )[ 0 ].keys( ) ) ), size = 14 )
ax.set_xlabel( "Genes", size = 18 )
ax.set_yticks( np.arange( 0.5, 74.5 ) )
ax.set_yticklabels( sorted( list( perSamplePerGeneUncalledCountUntreated.keys( ) ) ), size = 12 )
ax.set_ylabel( "Samples", size = 18 )
plt.setp( ax.get_xticklabels( ), rotation = 55, ha = "right", rotation_mode = "anchor")
plt.setp( ax.get_yticklabels( ), rotation = 0, ha = "right", rotation_mode = "anchor")
ax.set_title( "Percentage of Disregarded Repeat Positions per Sample and Gene\nRecoverMI not applied", size = 20 )
ax.figure.axes[-1].xaxis.label.set_size( 18 )
ax.figure.axes[-1].tick_params( labelsize = 14 )
fig.tight_layout( )
plt.show( )

### 3.6 Assess Uncalled Repeat Positions per Gene per Sample (with RecoverMI)

In [ ]:
# For each sample, assess the number of uncalled positions per gene and per repeat (for untreated .vcf).
perSamplePerGeneUncalledCountTreated = { }
for sampleName in os.listdir( "./R4_TPOMPeome_Hackl2022_VariantCalling/VariantCalling/" ) :
    perSamplePerGeneUncalledCountTreated[ sampleName ] = { }
    with open( "./R4_TPOMPeome_Hackl2022_VariantCalling/VariantCalling/" + sampleName + "/" + sampleName + "-NC_021490.2-UG-variants.fxd.vcf", "r" ) as treatedCalles :
        line = treatedCalles.readline( )
        while line :
            if not line.startswith( "#" ) :
                position = int( line.split( "\t" )[ 1 ] )
                for geneName in geneNames :
                    if geneName in perGeneRepeatPositions.keys( ) and not geneName in perSamplePerGeneUncalledCountTreated[ sampleName ] :
                        perSamplePerGeneUncalledCountTreated[ sampleName ][ geneName ] = 0
                    if geneName in perGeneRepeatPositions.keys( ) and position in perGeneRepeatPositions[ geneName ] and line.split( "\t" )[ 9 ].startswith( "./." ) :
                        perSamplePerGeneUncalledCountTreated[ sampleName ][ geneName ] += 1
            line = treatedCalles.readline( )

In [ ]:
from matplotlib.ticker import MultipleLocator

# Compute the percentage of uncalled positions per gene and per repeat and plot (for untreated .vcf).
fig, axs = plt.subplots( 4, 3, figsize = ( 11, 15 ), sharex = True, sharey = True )
axIdx0 = -1
axIdx1 = 0
for geneName in sorted( list( list( perSamplePerGeneUncalledCountTreated.values( ) )[ 0 ].keys( ) ) ) :
    axIdx0 += 1
    if axIdx0 > 3 :  
        axIdx0 = 0
        axIdx1 += 1
    percentages = [ ]
    ax = axs[ axIdx0 ][ axIdx1 ]
    for sampleName in sorted( list( perSamplePerGeneUncalledCountUntreated.keys( ) ) ) :
        geneInfo = annotatedGenes[ "TPANIC_" + geneName[ 2: ] ]
        genePositions = list( range( geneInfo[ 'locusStart' ], geneInfo[ 'locusEnd' ] + 1 ) )
        geneLength = geneInfo[ 'locusEnd' ] - geneInfo[ 'locusStart' ] + 1
        percentages.append(
            round( 100 * ( perSamplePerGeneUncalledCountTreated[ sampleName ][ geneName ] / geneLength ), 4 )
        )
    ax.hist( percentages, bins = [ v + 1*10**-6 for v in np.arange( -5, 105, 5 ) ], color = "dimgray", rwidth = 0.9 )
    ax.grid( )
    ax.set_title( geneName, size = 18 )
    if axIdx0 == 3 :
        ax.set_xticks( np.arange( 0, 120, 20 ) )
        ax.xaxis.set_tick_params( labelsize = 14 )
        ax.set_xlabel( "Disregarded Positions [%]", size = 18 )
    ax.xaxis.set_minor_locator(MultipleLocator(5))
    if axIdx1 == 0 :
        #ax.set_yticks( np.arange( 0.5, 74.5 ) )
        ax.yaxis.set_tick_params( labelsize = 14 )
        ax.set_ylabel( "Count", size = 18 )
plt.suptitle( "Histograms of Disregarded Repeat Positions in 74 Samples per Gene\nRecoverMI applied", y = 1.01, size = 20 )
plt.tight_layout( )
plt.show( )

In [ ]:
# Compute the percentage of uncalled positions per gene and per repeat and plot (for untreated .vcf).
sampleNames = set( )
geneNames = set( )
data2D = [ ]
for sampleName in sorted( list( perSamplePerGeneUncalledCountTreated.keys( ) ) ) :
    data1D = [ ]
    sampleNames.add( sampleName )
    for geneName in sorted( list( list( perSamplePerGeneUncalledCountTreated.values( ) )[ 0 ].keys( ) ) ) :
        geneInfo = annotatedGenes[ "TPANIC_" + geneName[ 2: ] ]
        genePositions = list( range( geneInfo[ 'locusStart' ], geneInfo[ 'locusEnd' ] + 1 ) )
        geneLength = geneInfo[ 'locusEnd' ] - geneInfo[ 'locusStart' ] + 1
        data1D.append(
            round( 100 * ( perSamplePerGeneUncalledCountTreated[ sampleName ][ geneName ] / geneLength ), 4 )
        )
        geneNames.add( geneName )
    data2D.append( data1D )

fig, ax = plt.subplots( figsize = ( 12, 18 ) )
res = sns.heatmap( data2D,
     cmap = "GnBu",
     cbar_kws = {
         "aspect": 50,
         "label": "Disregarded Positions (%)",
         "orientation": "horizontal",
         "pad": 0.08
     },
     vmin = 0,
     vmax = 100,
     center = 50,
     linewidths = 1,
     annot = True,
     annot_kws = {
         'va': 'center',
         'ha': 'center'
     }
)
for t in res.texts:
    if t.get_text( ) == "0" :
        t.set_text( "" )
    elif t.get_text( ) == "1e+02" :
        t.set_text( "100 %" )
    else :
        t.set_text( t.get_text( ) + " %" )
ax.set_xticklabels( sorted( list( list( perSamplePerGeneUncalledCountTreated.values( ) )[ 0 ].keys( ) ) ), size = 14 )
ax.set_xlabel( "Genes", size = 18 )
ax.set_yticks( np.arange( 0.5, 74.5 ) )
ax.set_yticklabels( sorted( list( perSamplePerGeneUncalledCountTreated.keys( ) ) ), size = 12 )
ax.set_ylabel( "Samples", size = 18 )
plt.setp( ax.get_xticklabels( ), rotation = 55, ha = "right", rotation_mode = "anchor")
plt.setp( ax.get_yticklabels( ), rotation = 0, ha = "right", rotation_mode = "anchor")
ax.set_title( "Percentage of Disregarded Repeat Positions per Sample and Gene\nRecoverMI applied", size = 20 )
ax.figure.axes[-1].xaxis.label.set_size( 18 )
ax.figure.axes[-1].tick_params( labelsize = 14 )
fig.tight_layout( )
plt.show( )